"Three missionaries and three cannibals are on the west bank of a river. They have a canoe that can hold two people, and they all must cross to the east bank of the river. There may never be more cannibals than missionaries on either side of the river, or the cannibals will eat the missionaries. Further, the canoe must have at least one person on board to cross the river. What sequence of crossings will successfully take the entire party across the river?"

- Like a maze, we also have a goal and we want to test if a state shows us that we have reached our goal. The goal test method would let us do that.

- We also want to make sure the state we're in is legal - ie. no more cannibals than missionaries on the same side.

- We would list out all the possible successors, and let __self.is_legal__ return whether a move is legal.

In [1]:
from __future__ import annotations
from typing import List, Optional
from generic_search import bfs, Node, node_to_path

MAX_NUM: int = 3

class MCState:
    def __init__(self, missionaries: int, cannibals: int, boat: bool) -> None:
        self.wm: int = missionaries # west bank missionaries
        self.wc: int = cannibals # west bank cannibals
        self.em: int = MAX_NUM - self.wm # east bank missionaries
        self.ec: int = MAX_NUM - self.wc # east bank cannibals
        self.boat: bool = boat # True means boat is on the left bank (after a move)
    
    def __str__(self) -> str:
        return(f'On the west bank there are {self.wm} missionaries and {self.wc} cannibals. \n'
              f'On the east bank there are {self.em} missionaries and {self.ec} cannibals. \n'
               f'Is the boat n the west bank? {self.boat}'            
              )
    def goal_test(self) -> bool:
        return self.is_legal and self.em == MAX_NUM and self.ec == MAX_NUM

    @property
    def is_legal(self) -> bool:
        if self.wm < self.wc and self.wm >0:
            return False
        if self.em < self.ec and self.em > 0:
            return False
        return True

    def successors(self) -> List[MCState]:
        sucs: List[MCState] = []
        if self.boat: # boat on west bank
            if self.wm > 1:
                sucs.append(MCState(self.wm - 2, self.wc, not self.boat))
            if self.wm > 0:
                sucs.append(MCState(self.wm - 1, self.wc, not self.boat))
            if self.wc > 1:
                sucs.append(MCState(self.wm, self.wc - 2, not self.boat))
            if self.wc > 0:
                sucs.append(MCState(self.wm, self.wc - 1, not self.boat))
            if (self.wc > 0) and (self.wm > 0):
                sucs.append(MCState(self.wm - 1, self.wc - 1, not self.boat))
        else: # boat on east bank
            if self.em > 1:
                sucs.append(MCState(self.wm + 2, self.wc, not self.boat))
            if self.em > 0:
                sucs.append(MCState(self.wm + 1, self.wc, not self.boat))
            if self.ec > 1:
                sucs.append(MCState(self.wm, self.wc + 2, not self.boat))
            if self.ec > 0:
                sucs.append(MCState(self.wm, self.wc + 1, not self.boat))
            if (self.ec > 0) and (self.em > 0):
                sucs.append(MCState(self.wm + 1, self.wc + 1, not self.boat))
        return [x for x in sucs if x.is_legal]

## Solving

In [2]:
def display_solution(path: List[MCState]):
    if len(path) == 0: #sanity check
        return
    old_state: MCState = path[0] 
    print(old_state)
    
    for current_state in path[1:]:
        if current_state.boat:
            print('{} missionaries and {} cannibals moved from the east bank to the westbank \n'\
                  .format(old_state.em - current_state.em, old_state.ec - current_state.ec))
        else:
            print('{} missionaries and {} cannibals moved from the west bank from the eastbank \n'\
            .format(old_state.wm - current_state.wm, old_state.wc - current_state.wc))
        print(current_state)
        old_state = current_state

In [3]:
start: MCState = MCState(MAX_NUM, MAX_NUM, True)
solution: Optional[Node[MCState]] = bfs(start, MCState.goal_test,
     MCState.successors)
if solution is None:
    print('No solution found!')
else:
    path: List[MCState] = node_to_path(solution)
    display_solution(path)

On the west bank there are 3 missionaries and 3 cannibals. 
On the east bank there are 0 missionaries and 0 cannibals. 
Is the boat n the west bank? True
0 missionaries and 2 cannibals moved from the west bank from the eastbank 

On the west bank there are 3 missionaries and 1 cannibals. 
On the east bank there are 0 missionaries and 2 cannibals. 
Is the boat n the west bank? False
0 missionaries and 1 cannibals moved from the east bank to the westbank 

On the west bank there are 3 missionaries and 2 cannibals. 
On the east bank there are 0 missionaries and 1 cannibals. 
Is the boat n the west bank? True
0 missionaries and 2 cannibals moved from the west bank from the eastbank 

On the west bank there are 3 missionaries and 0 cannibals. 
On the east bank there are 0 missionaries and 3 cannibals. 
Is the boat n the west bank? False
0 missionaries and 1 cannibals moved from the east bank to the westbank 

On the west bank there are 3 missionaries and 1 cannibals. 
On the east bank there